In [2]:
import time
from dask_image.imread import imread
from zarr.util import human_readable_size


animal = "DK59"
input_path = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/full_aligned'


start = time.time()
image_stack = imread(f"{input_path}/*.tif")
end = time.time()
print("Operation time: ", (end - start), "sec")

sections = image_stack.shape[0]
img_rows = image_stack.shape[1]
img_columns = image_stack.shape[2]

print(f"ORG IMAGE STACK MEM: {human_readable_size(image_stack.nbytes)}")
print(f"ORG IMAGE STACK SHAPE: {image_stack.shape}")
print(f"ORG IMAGE STACK DTYPE: {image_stack.dtype}")

Operation time:  39.09330177307129 sec
IMAGE STACK MEM: 1.5T
IMAGE STACK SHAPE: (388, 31465, 68157)
IMAGE STACK DTYPE: uint16


In [9]:
import os
import dask
from dask.distributed import LocalCluster, Client

# from dask_image.imread import imread
# import dask.array as da
# import zarr
# import lzma
# from numcodecs import LZMA
# import numpy as np


output_path = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/neuroglancer_data/zarr/C1.zarr'
tmp_dir = "/data/tmp"
max_client_ram = "20GB"
max_n_workers = 1

#CREATE 'LOCAL CLUSTER' TO PROCESS CHUNKS IN SERIAL MANNER
cluster = LocalCluster(
    n_workers=max_n_workers,
    threads_per_worker=1,
    memory_limit=max_client_ram,
    local_directory=tmp_dir,
    processes=False,
)

tiles = image_stack.rechunk('auto', balance=True)

print(f"[PROPOSED] ZARR IMG STACK SHAPE: {tiles.shape}")
print(f"[PROPOSED] ZARR IMG STACK CHUNKSIZE: {tiles.chunksize}")
print(f"[PROPOSED] ZARR IMG STACK INDIVIDUAL CHUNK MEMORY USAGE: {dask.config.get('array.chunk-size')}")
print(f"[PROPOSED] ZARR IMG STACK TOTAL CHUNK COUNT: {len(tiles.chunks[0])*len(tiles.chunks[1])*len(tiles.chunks[2])}")

/usr/local/share/pipeline/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36233 instead
  warnings.warn(


[PROPOSED] ZARR IMG STACK SHAPE: (388, 31465, 68157)
[PROPOSED] ZARR IMG STACK CHUNKSIZE: (1, 4495, 17040)
[PROPOSED] ZARR IMG STACK INDIVIDUAL CHUNK MEMORY USAGE: 128MiB
[PROPOSED] ZARR IMG STACK TOTAL CHUNK COUNT: 10864


In [10]:
tiles.to_zarr(output_path, overwrite=True)

2022-08-15 09:54:10,863 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 13.07 GiB -- Worker memory limit: 18.63 GiB
2022-08-15 09:54:10,875 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 13.09 GiB -- Worker memory limit: 18.63 GiB
2022-08-15 09:54:10,882 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

: 

: 

In [24]:
tiles

,Array,Chunk
Bytes,1.51 TiB,7.75 GiB
Shape,"(388, 31465, 68157)","(388, 3146, 3407)"
Count,,231 Chunks
Type,uint16,cupy.ndarray


2022-08-11 21:20:49,945 - distributed.worker_memory - WARNING - Worker tcp://127.0.0.1:46701 (pid=80861) exceeded 95% memory budget. Restarting...
2022-08-11 21:20:50,022 - distributed.nanny - WARNING - Restarting worker
2022-08-11 21:20:51,449 - distributed.worker - WARNING - Mismatched versions found

+---------+-------------+-----------+---------------------------+
| Package | This Worker | scheduler | workers                   |
+---------+-------------+-----------+---------------------------+
| dask    | 2022.8.0    | 2022.05.2 | {'2022.05.2', '2022.8.0'} |
+---------+-------------+-----------+---------------------------+


In [ ]:
import os
import dask.array as da
import zarr
#import lzma
#from numcodecs import LZMA

animal = 'DK59'
dest_output_path = "/data/dask_image.zarr"

store_dir = zarr.DirectoryStore(dest_output_path)
zarr_store = zarr.open(store_dir)
zarr_store.create_dataset(animal, shape=image_stack.shape, chunks=(sections, row_span, column_span), dtype=image_stack.dtype, compute=True, overwrite=True)
zarr.consolidate_metadata(dest_output_path)

#lzma_filters = [dict(id=lzma.FILTER_DELTA, dist=4), dict(id=lzma.FILTER_LZMA2, preset=1)]
#compressor = LZMA(filters=lzma_filters)

#z = zarr.array(image_stack, chunks=(sections, row_span, column_span))
#z.compressor

#da.to_zarr(image_stack, url=store_dir, compute=True, overwrite=True)

# group = zarr.hierarchy.group(store=store_dir, overwrite=True, synchronizer=zarr.ThreadSynchronizer())
# out_zarr1 = group.zeros(animal, shape=image_stack.shape, dtype=image_stack.dtype, chunks=[sections, row_span, column_span])
# out_zarr1[...] = image_stack[...]


# da.to_zarr(image_stack)


In [1]:
z.info

NameError: name 'z' is not defined

In [2]:
sys.path.append(os.path.abspath('/home/drinehart/preprocessing-pipeline/pipeline'))
from cell_extractor.ExampleFinder import create_examples_for_one_section
animal = 'DK59'

#step 2 - CREATE EXAMPLES
for threshold in [2100,2200,2300,2700]:#magic Yoav numbers
    #create_examples_for_all_sections(animal, disk = '/net/birdstore/Active_Atlas_Data/', segmentation_threshold=threshold, njobs=7)
    extractor = ExampleFinder(animal=animal, section=section, *args, **kwargs)
    # extractor.find_examples()
    # extractor.save_examples()



NameError: name 'ExampleFinder' is not defined

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print(type(image_stack[0]))
print(image_stack[0])

#thumb = image_stack[0].to_svg(500)
image_stack.compute_chunk_sizes()
# plt.imshow(image_stack[0], cmap='gray')
# plt.show()

<class 'dask.array.core.Array'>
dask.array<getitem, shape=(31465, 68157), dtype=uint16, chunksize=(31465, 68157), chunktype=cupy.ndarray>


: 

: 

In [21]:
import dask.array as darr
from dask.distributed import LocalCluster, Client
cluster = LocalCluster(
    n_workers=1,
    memory_limit="40GB",
    threads_per_worker=1,
)
client = Client(cluster)

dest_output_path = "/data/dask_image.zarr"

#a = darr.random.random(1e9, chunks='auto')
tiles.to_zarr(dest_output_path, overwrite=True)

/usr/local/share/pipeline/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39731 instead
  warnings.warn(
2022-08-11 16:14:29,683 - distributed.worker_memory - WARNING - Worker tcp://127.0.0.1:37427 (pid=76919) exceeded 95% memory budget. Restarting...
2022-08-11 16:14:29,695 - distributed.worker_memory - WARNING - Worker tcp://127.0.0.1:34191 (pid=78228) exceeded 95% memory budget. Restarting...
2022-08-11 16:14:29,738 - distributed.nanny - WARNING - Restarting worker
2022-08-11 16:14:29,818 - distributed.nanny - WARNING - Restarting worker
2022-08-11 16:14:30,836 - distributed.worker - WARNING - Mismatched versions found

+---------+-------------+-----------+----------+
| Package | This Worker | scheduler | workers  |
+---------+-------------+-----------+----------+
| dask    | 2022.8.0    | 2022.05.2 | 2022.8.0 |
+---------+-------------+-----------+----------+
2

KeyboardInterrupt: 